In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Valores históricos dos dados para o grupo 1
data = pd.read_excel('DadosG1.xlsx')
data["Valor"] = data["Valor"].str.replace(",",".")
data = data.reset_index(drop=True)
data["Data Coleta"] = pd.to_datetime(data["Data Coleta"], format="%d/%m/%Y")

cols = [
    "Valor",
    "Data Coleta",
    "Parametro",
    "Unidade"
]

params = [
    'pH',
    'Temperatura da Água',
    'Escherichia coli**',
    'Temperatura do Ar',
    'Turbidez',
    'Nitrogênio Kjeldahl',
    'Nitrogênio-Nitrito',
    'Nitrogênio-Nitrato',
    'Oxigênio Dissolvido',
    'Sólido Total',
    'Fósforo Total',
]

# Transforma os valores em float
data = data.loc[data['Parametro'].isin(params), cols]
data['Valor'] = data['Valor'].astype(float)

#####################################################################################################################################

# Cálculo do Nitrogênio Total
data_nitrogenio = data.loc[data['Parametro'].isin(['Nitrogênio Kjeldahl', 'Nitrogênio-Nitrito','Nitrogênio-Nitrato'])]

nitrogenio_list = data_nitrogenio.groupby(['Data Coleta'])['Valor'].sum()
nitrogenio = pd.DataFrame()
nitrogenio['Data Coleta'] = nitrogenio_list.index
nitrogenio['Valor'] = nitrogenio_list.values
nitrogenio['Parametro'] = 'Nitrogênio Total'
nitrogenio['Unidade'] = 'mg/L'

data.drop(data.loc[data['Parametro'].isin(['Nitrogênio Kjeldahl', 'Nitrogênio-Nitrito','Nitrogênio-Nitrato'])].index, inplace=True)
data = data.merge(nitrogenio, how='outer')


# Cálculo do delta de temperatura
delta_temperatura = data.loc[data['Parametro'].isin(['Temperatura da Água', 'Temperatura do Ar']), ['Data Coleta', 'Parametro', 'Valor', 'Unidade']]

df_pivot = delta_temperatura.pivot(index="Data Coleta", columns="Parametro", values="Valor")

df_pivot.columns = ["Temperatura da Água", "Temperatura do Ar"]
df_pivot["Delta Temperatura"] = df_pivot["Temperatura do Ar"] - df_pivot["Temperatura da Água"]
df_pivot['Unidade'] = '°C'
df_pivot['Parametro'] = 'Delta Temperatura'
df_pivot = df_pivot.rename(columns={'Delta Temperatura': 'Valor'})
df_pivot = df_pivot[['Parametro', 'Valor', 'Unidade']]
df_pivot.reset_index(inplace=True)

data.drop(data.loc[data['Parametro'].isin(['Temperatura da Água', 'Temperatura do Ar'])].index, inplace=True)
data = data.merge(df_pivot, how='outer')

# Cálculo dos Coliformes Termotolerantes
data.loc[data['Parametro'] == 'Escherichia coli**', 'Valor'] = data.loc[data['Parametro'] == 'Escherichia coli**', 'Valor'] * 1.25
data.loc[data['Parametro'] == 'Escherichia coli**', 'Parametro'] = 'Coliformes Termotolerantes'

# Junção final do DF
data.sort_values(['Parametro', 'Data Coleta'], inplace=True)
data = data[['Data Coleta', 'Parametro', 'Valor', 'Unidade']]

In [ ]:
data['Parametro'].unique()

In [ ]:
#Gráficos dos dados históricos:

plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(4, 2, figsize=(15, 15))
fig.suptitle('Parâmetros ao Longo do Tempo', size=16)

# Plot 1: pH
sns.lineplot(data=data[data['Parametro']=='pH'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[0,0])
axes[0,0].set_title('pH')
axes[0,0].set_ylabel('pH')

# Plot 2: Temperatura
temp_data = data[data['Parametro'].isin(['Delta Temperatura'])]
sns.lineplot(data=temp_data, x='Data Coleta', y='Valor', marker='o', ax=axes[0,1])
axes[0,1].set_title('Delta da temperatura (Ambiente x Água)')
axes[0,1].set_ylabel('°C')

# Plot 3: Coliformes (log scale due to large range)
sns.lineplot(data=data[data['Parametro']=='Coliformes Termotolerantes'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[1,0])
axes[1,0].set_yscale('log')
axes[1,0].set_title('Coliformes Termotolerantes')
axes[1,0].set_ylabel('UFC/100mL')

# Plot 4: Nitrogênio Total
sns.lineplot(data=data[data['Parametro']=='Nitrogênio Total'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[1,1])
axes[1,1].set_title('Nitrogênio Total')
axes[1,1].set_ylabel('mg/L')

# Plot 5: Turbidez
sns.lineplot(data=data[data['Parametro']=='Turbidez'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[2,0])
axes[2,0].set_title('Turbidez')
axes[2,0].set_ylabel('UNT')

# Plot 6: Oxigênio Dissolvido
sns.lineplot(data=data[data['Parametro']=='Oxigênio Dissolvido'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[2,1])
axes[2,1].set_title('Oxigênio Dissolvido')
axes[2,1].set_ylabel('mg/L')

# Plot 7: Fósforo total
sns.lineplot(data=data[data['Parametro']=='Fósforo Total'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[3,0])
axes[3,0].set_title('Fósforo Total')
axes[3,0].set_ylabel('mg/L')

# Plot 8: Fósforo total
sns.lineplot(data=data[data['Parametro']=='Sólido Total'], 
            x='Data Coleta', y='Valor', 
            marker='o', ax=axes[3,1])
axes[3,1].set_title('Sólido Total')
axes[3,1].set_ylabel('mg/L')


plt.tight_layout()
plt.show()

In [ ]:
# Cálculo do IQA (Índice de Qualidade da Água)
iqa_file = pd.ExcelFile('DadosIQA.xlsx')

# Relação dos arquivos de base para cálcudo do IQA com a nossa tabela criada
iqa_data = {
    'pH': {
        'name': 'pH',
        'lim_inf': [2, 2],
        'lim_sup': [12, 3],
        'q': [],
        'peso': 0.12
    },
    'Delta Temperatura': {
        'name': 'Temperatura',
        'lim_inf': [-5, np.inf],
        'lim_sup': [15, 9],
        'q': [],
        'peso': 0.10
    },
    'Coliformes Termotolerantes': {
        'name': 'Coliformes',
        'lim_inf': [1, 100],
        'lim_sup': [1e5, 3],
        'q': [],
        'peso': 0.15
    },
    'Nitrogênio Total': {
        'name': 'N',
        'lim_inf': [0.5, 100],
        'lim_sup': [100, 1],
        'q': [],
        'peso': 0.10
    },
    'Turbidez': {
        'name': 'Turbidez',
        'lim_inf': [0.5, 100],
        'lim_sup': [100, 5],    
        'q': [],
        'peso': 0.08
    },
    'Oxigênio Dissolvido': {
        'name': 'OD',
        'lim_inf': [5, 5],
        'lim_sup': [140, 47],
        'q': [],
        'peso': 0.17
    },
    'Fósforo Total': {
        'name':'Fosforo',
        'lim_inf': [0.5, 100],
        'lim_sup': [10, 5],
        'q': [],
        'peso': 0.10
    },
    'Sólido Total': {
        'name': 'Residuos',
        'lim_inf': [0, 83],
        'lim_sup': [500, 32],
        'q': [],
        'peso': 0.08
    }
}

lista_q = []

# Loop para calcular os Q's de cada parâmetro
for param in data['Parametro'].unique():
    data_per_param = data.loc[data['Parametro'] == param]
    iqa_df = pd.read_excel(iqa_file, iqa_data[param]['name'])
    
    for val in data_per_param['Valor']:
        if val < iqa_data[param]['lim_inf'][0]:
            q = iqa_data[param]['lim_inf'][1]
        elif val > iqa_data[param]['lim_sup'][0]:
            q = iqa_data[param]['lim_sup'][1]
        else:
            for index in range(1, len(iqa_df[iqa_df.columns[0]])):
                if val <= iqa_df[iqa_df.columns[0]][index]:
                    q = np.interp(val, 
                                    [float(iqa_df[iqa_df.columns[0]][index]), float(iqa_df[iqa_df.columns[0]][index-1])], 
                                    [iqa_df[iqa_df.columns[1]][index], iqa_df[iqa_df.columns[1]][index-1]])
                    break

        iqa_data[param]['q'].append(float(q))

for param in iqa_data.keys():
    print(f'Valores de q para {param}: {iqa_data[f"{param}"]["q"]}')

In [ ]:
# Pesos dos parâmetros
pesos = np.array([iqa_data[param]['peso'] for param in iqa_data])

# Valores de qualidade (q) para cada amostra
q_values = np.array([iqa_data[param]['q'] for param in iqa_data])

IQA = np.prod(q_values**pesos[:, None], axis=0)

IQA